# Dataset initialization and transformation

In [1]:
import pandas as pd

In [2]:
# convert datasets to csv from txt

# \t used as separator, because of raw data format and the headers as row 0
products_s = pd.read_csv('../datasets/stock_forecasting/raw/2022-2025/[LT] Products [STOCK].txt', sep='\t', header=0) # stock
tickets_c_i = pd.read_csv('../datasets/stock_forecasting/raw/2022-2025/[LT] Tickets [CUSTOM] [ITEMS].txt', sep='\t', header=0) # customer order items
tickets_c_m = pd.read_csv('../datasets/stock_forecasting/raw/2022-2025/[LT] Tickets [CUSTOM] [MAIN].txt', sep='\t', header=0) # customer order main data
sp_inv_adds = pd.read_csv('../datasets/stock_forecasting/raw/2022-2025/[LT] SP Inventory [ADDS].txt', sep='\t', header=0) # customer order main data
sp_inv_rel = pd.read_csv('../datasets/stock_forecasting/raw/2022-2025/[LT] SP Inventory [REL].txt', sep='\t', header=0) # customer order main data


/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_15600/2542517297.py:4: DtypeWarning: Columns (4,6,20,50,52) have mixed types. Specify dtype option on import or set low_memory=False.
  products_s = pd.read_csv('../datasets/stock_forecasting/raw/2022-2025/[LT] Products [STOCK].txt', sep='\t', header=0) # stock
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_15600/2542517297.py:5: DtypeWarning: Columns (5,29,42,48) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_i = pd.read_csv('../datasets/stock_forecasting/raw/2022-2025/[LT] Tickets [CUSTOM] [ITEMS].txt', sep='\t', header=0) # customer order items
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_15600/2542517297.py:6: DtypeWarning: Columns (43,57,106,141,146,211,212,247,255,294,298,316,327,332,350,354) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_m = pd.read_csv('../datasets/stock_forecasting/raw/2022-2025/[LT] Tickets [CUSTOM]

In [3]:
# display the first 5 rows of the datasets
print('-------------- Product [STOCK] ----------------')
print(products_s.head())

print('-------------- Tickets [CUSTOM] [ITEMS] ----------------')
print(tickets_c_i.head())

print('-------------- Tickets [CUSTOM] [MAIN] ----------------')
print(tickets_c_m.head())
print('------------------------------')


-------------- Product [STOCK] ----------------
   Adhesive  Alternate  Available  BackOrdered Box_Size  CaseQty Color  \
0       NaN        NaN          0            0      NaN      NaN   NaN   
1       NaN        NaN          0            0      NaN      NaN   NaN   
2       NaN        NaN          0            0      NaN      NaN   NaN   
3       NaN        NaN          0            0      NaN      NaN   NaN   
4       NaN        NaN          0            0      NaN      NaN   NaN   

   Commission   Cost  Currency_ExchangeRate  ...  SupplierName  SupplierNo  \
0           0   0.00                      0  ...           NaN         NaN   
1           0   0.00                      0  ...           NaN         NaN   
2           0   0.00                      0  ...           NaN         NaN   
3           0   0.00                      0  ...           NaN         NaN   
4           0  49.11                      0  ...           NaN         NaN   

  SupplierNotes SupplierPartNo Tag Tot

In [4]:
# change product stock column 'ProductNo' to 'ProductNumber' to match the other datasets
products_s.rename(columns={'ProductNo': 'ProductNumber'}, inplace=True)

# change the column name of 'Number' to 'TicketNumber'
tickets_c_m.rename(columns={'Number': 'TicketNumber'}, inplace=True)

In [5]:
tickets_c_m.head()

,Act_MakeReady_Footage,Act_OTHER_Hours,ActArtwork,ActFinMaterial,ActFootage,ActMRHrs,ActPackHrs,ActPostPressHours,ActPressSpd,ActQuantity,...,TotalOrderWeight,TotalShipWeight,TurnBar,UL,updateTimeDateStamp,Use_TurretRewinder,UserDef_MR_1,UserDef_MR_1_Lb,UserDef_MR_2,UserDef_MR_2_Lb
0,0,0,0,0,0,0.0,0,0,0,0,...,0,0,False,False,2025-02-18T13:13:14.001Z,False,False,NaN,False,NaN
1,0,0,0,0,0,0.0,0,0,0,0,...,0,0,False,False,2025-02-18T13:10:42.128Z,False,False,NaN,False,NaN
2,0,0,0,0,0,0.0,0,0,0,0,...,0,0,False,False,2025-02-18T09:59:34.975Z,False,False,NaN,False,NaN
3,0,0,0,0,0,0.0,0,0,0,0,...,0,0,False,False,2025-02-17T22:32:37.678Z,False,False,NaN,False,NaN
4,0,0,0,0,0,0.0,0,0,0,0,...,0,0,False,False,2025-02-17T22:31:37.158Z,False,False,NaN,False,NaN


In [6]:
# merge the datasets
merged_data = pd.merge(tickets_c_m, tickets_c_i, on='TicketNumber')
merged_data = pd.merge(merged_data, products_s, on='ProductNumber')

In [7]:
print('----- DROP BOOLEAN AND NULL COLUMNS (NOT NEEDED) -----\n')

# drop columns with all rows as missing values (NaN/0)
columns_to_drop = merged_data.columns[merged_data.isnull().all()].tolist()
print(f'Merged Data - Columns to be dropped (Missing Vals): {columns_to_drop}\n')
merged_data = merged_data.dropna(axis=1, how='all')

# set the name attribute for each DataFrame
merged_data.name = 'Merged Data'

# drop columns with all rows containing the same values of either 0, True, or False
for df in [merged_data]:
    columns_to_drop = [col for col in df.columns if df[col].nunique() == 1 and df[col].iloc[0] in [0, True, False]]
    print(f'{df.name} - Columns to be dropped (All cols with rows same vals): {columns_to_drop}\n')
    df.drop(columns=columns_to_drop, inplace=True)

print('-------------------------------------------------------\n')

----- DROP BOOLEAN AND NULL COLUMNS (NOT NEEDED) -----

Merged Data - Columns to be dropped (Missing Vals): ['BackStage_DefaultReportForm', 'BackStage_SmartMarkSet', 'BillCounty', 'CoreType', 'EndUserNum', 'EndUserPO', 'Equip3_ID', 'Equip3_Status', 'Est_v_Act_Notes', 'Ink_Status', 'JDF_Note_to_DFE', 'JDF_Send_Msg', 'MiscChargeDesc1', 'MiscChargeDesc2', 'MiscChargeDesc3', 'MiscChargeDesc4', 'PlateStat', 'ProofStat', 'Roto_CEL_Product_ID', 'Roto_Quote_Line_ID', 'Roto_Quote_Number', 'Schedule_Status', 'ShipAttn_EmailAddress', 'ShipCounty', 'ShipStat', 'Tag_x', 'Tool3Descr', 'Tool4Descr', 'Tool5Descr', 'ToolNo3', 'ToolNo4', 'ToolNo5', 'ToolStat', 'UserDef_MR_1_Lb', 'UserDef_MR_2_Lb', 'Assigned', 'Desc2_x', 'ediLineNumber', 'eTraxx_Customer_Notes', 'Location_x', 'PO_Number', 'Adhesive', 'Alternate', 'eTraxx_Forecast_Range', 'Material', 'Tag_y', 'UPC']

Merged Data - Columns to be dropped (All cols with rows same vals): ['Act_OTHER_Hours', 'ActArtwork', 'ActFinMaterial', 'ActPostPressHours',

In [8]:
# drop 'TicQuantity' column as it is for a order (where orders can include multiple products)
merged_data.drop(columns='TicQuantity', inplace=True)
print('Dropped TicQuantity column\n')

Dropped TicQuantity column



In [9]:
# drop rows with '0' values in 'OrderQuantity' column
merged_data = merged_data[merged_data.OrderQuantity != 0]

# display a confirmation message with the rows dropped count
print(f'Dropped rows with 0 values in OrderQuantity column. Rows dropped: {len(merged_data[merged_data.OrderQuantity == 0])}\n')

Dropped rows with 0 values in OrderQuantity column. Rows dropped: 0



In [10]:
# display merged data after dropping columns
print('-------------- MERGED DATA ----------------')
print(merged_data.head())
# display ALL columns
print(merged_data.columns)

# Write column names to a text file
with open('columns.txt', 'w') as f:
    for col in merged_data.columns:
        f.write(f"{col}\n")

print('-------------------------------------------')

# print all customername and suppliername which arent null in seperate dfs
# print('-------------- CUSTOMER NAMES ----------------')
# print(merged_data[merged_data.CustomerName.notnull()]['CustomerName'].unique())
# print('----------------------------------------------')

print('-------------- SUPPLIER NAMES ----------------')
print(merged_data[merged_data.SupplierName.notnull()]['SupplierName'].unique())
print('----------------------------------------------')

-------------- MERGED DATA ----------------
   Act_MakeReady_Footage  ActFootage  ActMRHrs  ActPackHrs  ActPressSpd  \
0                      0           0       0.0           0            0   
1                      0           0       0.0           0            0   
2                      0           0       0.0           0            0   
3                      0           0       0.0           0            0   
4                      0           0       0.0           0            0   

   ActQuantity  ActRunHrs  ActStockCost  ActTotalCost  \
0            0        0.0           0.0        356.25   
1            0        0.0           0.0        133.95   
2            0        0.0           0.0          5.70   
3            0        0.0           0.0          5.00   
4            0        0.0           0.0        347.70   

   ActualBillings_NetOfSalesTax  ...   ProdSubClass  Product_Image_Size  \
0                           0.0  ...  Self Adhesive                 0.0   
1           

In [11]:
import numpy as np
import pandas as pd

'''
TRYING DIFFERENT FEATURES TO SEE HOW THEY ACT WITH MODEL PERFORMANCE
'''

# ensure date columns are properly formatted
merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'], errors='coerce')
merged_data['Ship_by_Date'] = pd.to_datetime(merged_data['Ship_by_Date'], errors='coerce')
merged_data['order_date'] = merged_data['OrderDate']
merged_data['ship_by_date'] = merged_data['Ship_by_Date']

# time-based features
merged_data['order_month'] = merged_data['order_date'].dt.month
merged_data['order_week'] = merged_data['order_date'].dt.isocalendar().week
merged_data['order_year'] = merged_data['order_date'].dt.year
merged_data['order_weekday'] = merged_data['OrderDate'].dt.weekday
merged_data['is_weekend'] = (merged_data['order_weekday'] >= 5).astype(int)
merged_data['quarter'] = merged_data['OrderDate'].dt.quarter
merged_data['is_end_of_month'] = (merged_data['OrderDate'].dt.day > 25).astype(int)

# year-over-year growth (yoy)
merged_data['prev_year_sales'] = merged_data.groupby('ProductNumber')['OrderQuantity'].shift(12)
merged_data['prev_week_sales'] = merged_data.groupby('ProductNumber')['OrderQuantity'].shift(1)
merged_data['yoy_growth'] = (merged_data['OrderQuantity'] - merged_data['prev_year_sales']) / merged_data['prev_year_sales']

merged_data['sales_2022'] = merged_data.apply(lambda x: x['OrderQuantity'] if x['order_year'] == 2022 else 0, axis=1)
merged_data['sales_2023'] = merged_data.apply(lambda x: x['OrderQuantity'] if x['order_year'] == 2023 else 0, axis=1)
merged_data['sales_2024'] = merged_data.apply(lambda x: x['OrderQuantity'] if x['order_year'] == 2024 else 0, axis=1)

# growth of product sales per year (%)
merged_data['growth_2023'] = (merged_data['sales_2023'] - merged_data['sales_2022']) / merged_data['sales_2022'] * 100
merged_data['growth_2024'] = (merged_data['sales_2024'] - merged_data['sales_2023']) / merged_data['sales_2023'] * 100
 

# lag features (considers past trends via products)
merged_data['prev_month_sales'] = merged_data.groupby('ProductNumber')['OrderQuantity'].shift(1)
merged_data['prev_2_month_sales'] = merged_data.groupby('ProductNumber')['OrderQuantity'].shift(2)
merged_data['prev_3_month_sales'] = merged_data.groupby('ProductNumber')['OrderQuantity'].shift(3)

# time difference features
# merged_data['days_since_last_order'] = merged_data.groupby('ProductNumber')['OrderDate'].diff().dt.days.fillna(30)

# rolling features
merged_data['moving_avg_3m'] = merged_data.groupby('ProductNumber')['OrderQuantity'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
merged_data['moving_avg_6m'] = merged_data.groupby('ProductNumber')['OrderQuantity'].transform(lambda x: x.rolling(window=6, min_periods=1).mean())
merged_data['moving_avg_12m'] = merged_data.groupby('ProductNumber')['OrderQuantity'].transform(lambda x: x.rolling(window=12, min_periods=1).mean())
merged_data['moving_avg_18m'] = merged_data.groupby('ProductNumber')['OrderQuantity'].transform(lambda x: x.rolling(window=18, min_periods=1).mean())

merged_data['var_1m'] = merged_data.groupby('ProductNumber')['OrderQuantity'].transform(lambda x: x.rolling(window=1, min_periods=1).var())
merged_data['var_3m'] = merged_data.groupby('ProductNumber')['OrderQuantity'].transform(lambda x: x.rolling(window=3, min_periods=1).var())
merged_data['var_6m'] = merged_data.groupby('ProductNumber')['OrderQuantity'].transform(lambda x: x.rolling(window=6, min_periods=1).var())
merged_data['var_12m'] = merged_data.groupby('ProductNumber')['OrderQuantity'].transform(lambda x: x.rolling(window=12, min_periods=1).var())
merged_data['var_18m'] = merged_data.groupby('ProductNumber')['OrderQuantity'].transform(lambda x: x.rolling(window=18, min_periods=1).var())

# log-transformed variance
merged_data['log_var_1m'] = np.log1p(merged_data['var_1m'])  # log1p prevents log(0) errors
merged_data['log_var_3m'] = np.log1p(merged_data['var_3m'])
merged_data['log_var_6m'] = np.log1p(merged_data['var_6m'])
merged_data['log_var_12m'] = np.log1p(merged_data['var_12m'])
merged_data['log_var_18m'] = np.log1p(merged_data['var_18m'])

# check and fill missing values
for col in [
    'prev_month_sales', 'prev_week_sales', 
    'moving_avg_3m', 'moving_avg_6m', 'moving_avg_12m', 'moving_avg_18m',
    'prev_2_month_sales', 'prev_3_month_sales',
    'var_1m', 'var_3m', 'var_6m', 'var_12m', 'var_18m'
    ]:
    merged_data[col] = merged_data[col].fillna(merged_data[col].mean())

# product lifestyle (upcoming, declining, mature) - REDUCES METRICS AND PREDICTION ACCURACIES
# merged_data['product_lifecycle'] = merged_data.groupby('ProductNumber')['OrderQuantity'].transform(lambda x: np.where(x.rolling(window=12, min_periods=1).mean() > x.mean(), 'mature', 'new'))

# # interaction Features
# merged_data['interaction_1'] = merged_data['prev_month_sales'] * merged_data['var_12m']
# merged_data['interaction_2'] = merged_data['prev_week_sales'] * merged_data['var_12m']
# merged_data['interaction_3'] = merged_data['moving_avg_3m'] * merged_data['moving_avg_12m']
# merged_data['interaction_4'] = merged_data['prev_2_month_sales'] * merged_data['var_18m']
# merged_data['interaction_5'] = merged_data['prev_3_month_sales'] * merged_data['var_18m']

# # demand Factors
# merged_data['inventory_ratio'] = merged_data['PhysicalInv'] / (merged_data['OnOrder'] + 1)
# merged_data['is_backordered'] = merged_data['BackOrdered'].notna().astype(int)
# merged_data['customer_order_count'] = merged_data.groupby('Customer_Num')['OrderQuantity'].transform('count')
# merged_data['customer_avg_order'] = merged_data.groupby('Customer_Num')['OrderQuantity'].transform('mean')

# aggregation
product_sales = merged_data.groupby([ # group rows by:
    'ProductNumber', 
    'order_year', 'order_month', 'order_week', 'order_weekday', 'is_weekend',
    'Customer_Num'
]).agg({ # include these columns with respective data 
    'OrderQuantity': 'sum', 
    'prev_month_sales': 'mean',
    'prev_week_sales': 'mean',
    'prev_2_month_sales': 'mean',
    'prev_3_month_sales': 'mean',
    'var_1m': 'mean',
    'var_3m': 'mean',
    'var_6m': 'mean',
    'var_12m': 'mean',
    'var_18m': 'mean',
    'log_var_1m': 'mean',
    'log_var_3m': 'mean',
    'log_var_6m': 'mean',
    'log_var_12m': 'mean',
    'log_var_18m': 'mean',
    'yoy_growth': 'mean',
    'moving_avg_3m': 'mean',
    'moving_avg_6m': 'mean',
    'moving_avg_12m': 'mean',
    'moving_avg_18m': 'mean',
    'sales_2022': 'sum',
    'sales_2023': 'sum',
    'sales_2024': 'sum',
    'growth_2023': 'mean',
    'growth_2024': 'mean',
    }).reset_index()


/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_15600/707283807.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'], errors='coerce')
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_15600/707283807.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'], errors='coerce')
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_15600/707283807.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. 

In [12]:
# add the more feature columns to the product_sales dataset
product_sales = pd.merge(product_sales, products_s[['ProductNumber', 'PhysicalInv']], on='ProductNumber')

# sales data for SARIMAX time series model test
# sales_data = merged_data.groupby(['OrderDate'])['OrderQuantity'].sum()

sales_data = merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'])

# 'OrderDate' to datetime
merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'])

# index correctly
sales_data = merged_data.set_index(['OrderDate', 'ProductNumber'])
sales_data = sales_data['OrderQuantity']

/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_15600/1541869033.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data = merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'])
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_15600/1541869033.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'])


In [13]:
product_sales.head(25)

,ProductNumber,order_year,order_month,order_week,order_weekday,is_weekend,Customer_Num,OrderQuantity,prev_month_sales,prev_week_sales,...,moving_avg_3m,moving_avg_6m,moving_avg_12m,moving_avg_18m,sales_2022,sales_2023,sales_2024,growth_2023,growth_2024,PhysicalInv
0,AFS-005-0002Z,2022,4,14,0,0,196.0,10000,110031.53337,110031.53337,...,10000.000000,10000.000000,10000.000000,10000.000000,10000,0,0,-100.0,NaN,0
1,AFS-005-0003S,2024,5,20,1,0,196.0,10000,10000.00000,10000.00000,...,10000.000000,9375.000000,9375.000000,9375.000000,0,0,10000,NaN,inf,2500
2,AFS-005-0003S,2024,9,38,4,0,196.0,10000,10000.00000,10000.00000,...,9166.666667,9166.666667,9166.666667,9166.666667,0,0,10000,NaN,inf,2500
3,AFS-005-0003S,2024,10,41,3,0,196.0,10000,7500.00000,7500.00000,...,8750.000000,8750.000000,8750.000000,8750.000000,0,0,10000,NaN,inf,2500
4,AFS-005-0003S,2025,1,5,3,0,196.0,7500,110031.53337,110031.53337,...,7500.000000,7500.000000,7500.000000,7500.000000,0,0,0,NaN,NaN,2500
5,AFS-005-0003U,2023,9,39,4,0,196.0,2500,10000.00000,10000.00000,...,10833.333333,10833.333333,10833.333333,10833.333333,0,2500,0,inf,-100.0,0
6,AFS-005-0003U,2023,11,45,3,0,196.0,10000,20000.00000,20000.00000,...,15000.000000,15000.000000,15000.000000,15000.000000,0,10000,0,inf,-100.0,0
7,AFS-005-0003U,2023,12,52,3,0,196.0,20000,110031.53337,110031.53337,...,20000.000000,20000.000000,20000.000000,20000.000000,0,20000,0,inf,-100.0,0
8,AFS-005-0003V,2023,8,33,2,0,196.0,2500,110031.53337,110031.53337,...,2500.000000,2500.000000,2500.000000,2500.000000,0,2500,0,inf,-100.0,0
9,AFS-005-0003W,2023,6,24,0,0,196.0,2500,110031.53337,110031.53337,...,2500.000000,2500.000000,2500.000000,2500.000000,0,2500,0,inf,-100.0,0


In [14]:
# saves outputs of products_sales to '../datasets/stock_forecasting/final'
product_sales.to_csv('../datasets/stock_forecasting/final/product_sales.csv', index=False)

In [15]:
sales_data.head()

OrderDate   ProductNumber
2025-02-18  PLA-005-0142Z    338000
            PLA-005-0144Y    156000
            ODO-032-0067Y     43750
2025-02-17  DAK-026-0021Y     12000
            DAK-026-0003X     60000
Name: OrderQuantity, dtype: int64

In [16]:
# extract all columns from all datasets and save them to a txt file (products_s, tickets_c_i, tickets_c_m)
# extract columns from each dataset and save them to a txt file one by one
columns = pd.DataFrame()

# concatenate columns from products_s
columns = pd.concat([columns, pd.DataFrame(products_s.columns, columns=['Column'])])

# concatenate columns from tickets_c_i
columns = pd.concat([columns, pd.DataFrame(tickets_c_i.columns, columns=['Column'])])

# concatenate columns from tickets_c_m
columns = pd.concat([columns, pd.DataFrame(tickets_c_m.columns, columns=['Column'])])

# concatenate columns from sp_inv_adds
columns = pd.concat([columns, pd.DataFrame(sp_inv_adds.columns, columns=['Column'])])

# concatenate columns from sp_inv_rel
columns = pd.concat([columns, pd.DataFrame(sp_inv_rel.columns, columns=['Column'])])

# save the columns to a txt file
# columns.to_csv('../datasets/stock_forecasting/2022-2025/columns.txt', index=False)